In [1]:
# libraries
import psycopg2 as pg
import pandas as pd
import numpy as np
import os

# set environment variable for psycog2 (for some systems)
os.environ["PGGSSENCMODE"] = "disable"

# get the stored passwords
f=open("credentials_david.txt", "rt")
pwd=f.readline().strip()  #di   password
hpwd=f.readline().strip() #home password
f.close()

In [2]:
# di test: WORKING
#host_di = "appserver-01.alunos.di.fc.ul.pt"
#db_di = "tpd012"
#user_di = "tpd012"

#conn = pg.connect(host = host_di,
#                  database = db_di,
#                  user = user_di,
#                  password = pwd)
#conn.close()

# local test: WORKING
host_local = "localhost"
db_local = "tpd012"
user_local = "postgres"

conn = pg.connect(host = host_local,
                  database = db_local,
                  user = user_local,
                  password=hpwd)
conn.close()

In [3]:
# functions
def run_sql_command(sql, host, database, user, password):
    """Executes a single SQL statement from a string variable and the database credentials"""
    conn = pg.connect(host = host,
                      database = database,
                      user = user,
                      password = password)
    cur = conn.cursor()
    cur.execute(sql)
    cur.close()
    conn.commit()
    conn.close() 
    
def intTryParse(value):
    """Parse a string to an integer"""
    try:
        a = int(value)
        return True
    except ValueError:
        return False

In [54]:
listings_al_file_path = '../data/listings_al.csv'
df_listings_al = pd.read_csv(listings_al_file_path)

C:\Users\david\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (61,62) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## 2.2. Preprocessing

The defined _'Property'_ dimension table in __Phase I__ is as follows:

<img src="property_schema.png" width="150" align="center"/>

In [48]:
def delete_null_rows(df,columns):
    """Removes all"""
    processed_df = df[columns].copy()
    total_rows = processed_df.shape[0]
    delete_rows = []
    
    if processed_df.isnull().values.any(): # if there are any null values in DataFrame, process DataFrame
        for index, row in enumerate(processed_df.itertuples(), start = 0):
            if (pd.Series(row).isnull().values.any()): # if row has any null value
                delete_rows.append(index) # add row index to delete list
    
    processed_df.drop(df.index[delete_rows], inplace = True) # delete rows fr
    processed_df = processed_df.reset_index().drop('index', axis = 1)
    print('Deleted {} rows ({}% of total rows)'.format(len(delete_rows), round(len(delete_rows)*100/total_rows, 2)))
    return processed_df

In [55]:
df_columns = ['id',
              'property_type',
              'room_type',
              'accommodates',
              'bathrooms',
              'bedrooms',
              'beds',
              'bed_type']

df_property = delete_null_rows(df_listings_al,df_columns)

Deleted 71 rows (0.41% of total rows)


In [56]:
df_property.head(65)

,id,property_type,room_type,accommodates,bathrooms,bedrooms,beds,bed_type
0,25659,Apartment,Entire home/apt,3,1.0,1.0,1.0,Real Bed
1,29248,Apartment,Entire home/apt,4,1.0,1.0,2.0,Real Bed
2,29396,Apartment,Entire home/apt,4,1.0,1.0,1.0,Real Bed
3,29720,Apartment,Entire home/apt,16,8.0,9.0,14.0,Real Bed
4,27899698,Apartment,Entire home/apt,4,2.0,2.0,2.0,Real Bed
...,...,...,...,...,...,...,...,...
60,174064,Apartment,Entire home/apt,6,2.0,3.0,4.0,Real Bed
61,178515,Apartment,Entire home/apt,6,2.0,3.0,3.0,Real Bed
62,2561894,Apartment,Entire home/apt,4,2.0,2.0,2.0,Real Bed
63,5753854,Apartment,Entire home/apt,5,2.0,3.0,3.0,Real Bed


In [52]:


# change 'property_type' to four categories, exclude exceptions
for i, value in enumerate(df_property['property_type']):
    if value in ["Apartment", "Serviced apartment", "Aparthotel","Loft"]:  
        df_property.loc[i, 'property_type'] = 'Apartment'
    elif value in ["House","Townhouse","Villa","Dome house","Vacation home","Lighthouse","Casa particular (Cuba)","Tiny house","Farm stay","Cottage"]:
        df_property.loc[i, 'property_type'] = 'House'
    elif value in ["Guesthouse","Guest suite"]:
        df_property.loc[i, 'property_type'] = 'Guesthouse'
    elif value in ["Hostel","Bed and breakfast","Boutique hotel","Hotel"]:
        df_property.loc[i, 'property_type'] = 'Hotel/Hostel'
    else:
    delete_rows.append(i)

In [ ]:
# keep only columns which are relevant for Property dimension


# removing rows with missing data
kept_data = df_result[df_columns].dropna().reset_index().copy()
print('Removing rows with missing data keeps {}% of the original data.'.format( round( (kept_data.shape[0] / df_result[df_columns].shape[0]) * 100, 2) ) )

In [ ]:
property = {}
property['PK'] = []
property['PropertyType'] = []
property['RoomType'] = []
property['Accommodates'] = []
property['Bathrooms'] = []
property['Bedrooms'] = []
property['Beds'] = []
property['BedType'] = []


### 2.2.?. Creating _Property_ dimension table in SQL

The SQL statement to create the table is as follows:

In [ ]:
create_table = """
CREATE TABLE Property (
    PROPERTY_ID SERIAL PRIMARY KEY NOT NULL,
    PROPERTY_TYPE VARCHAR(30) CHECK (PROPERTY_TYPE in ('Apartment', 'GuestHouse', 'House', 'Hostel', 'Room')) NOT NULL,
    ROOM_TYPE VARCHAR(30) CHECK (ROOM_TYPE in ('Entire Property', 'Private Room', 'Hotel Room', 'Shared Room')) NOT NULL,
    ACCOMMODATES VARCHAR(10) CHECK (ACCOMMODATES in ('0-2','2-4','4-6','>6')) NOT NULL,
    BATHROOMS VARCHAR(10) CHECK (BATHROOMS in ('0','1','2','3','>=4')) NOT NULL,
    BEDROOMS VARCHAR(10) CHECK (BEDROOMS in ('T0','T1','T2','T3','T>=4')) NOT NULL,
    BEDS VARCHAR(10) CHECK (BATHROOMS in ('0','1','2','3','>=4')) NOT NULL,
    BED_TYPE VARCHAR(30) CHECK (BED_TYPE in ('Real Bed', 'Pull-out Sofa', 'Futton', 'Couch', 'Airbed')) NOT NULL,
    PRICE_SRQT VARCHAR(10) CHECK (PRICE_SRQT in ('Expensive', 'Medium', 'Cheap')) NOT NULL
)
"""

In [ ]:
# creating table
run_sql_command(create_table,
                host_local,
                db_local,
                user_local,
                hpwd)

The `create_table` command contains the integrity constraints essential for modelling the dimension.

The dimension `Property` is ready for data __loading__. Now, it's necessary to __transform__ the data, which was previously __extracted__ from its source.

# Drafts

This will change in future, with the addition of price/sqr. meter